In [262]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from collections import Counter

sc = pyspark.SparkContext.getOrCreate()
sql_c = SQLContext(sc)

In [331]:
s = "The quick brown fox jumps over the lazy dog and the fox was very happy"
word = s.lower().split(' ')
freq = [dict(Counter(w))  for w in word]
tuples = []
for f in freq:
    for k,v in f.iteritems():
        tuples.append((k,v))
r = sc.parallelize(tuples)
r.take(2)
letter_count = r.reduceByKey(lambda x, y: x+y)
print(letter_count.take(5))

[('a', 4), ('e', 5), ('i', 1), ('m', 1), ('q', 1)]


In [330]:
grp_k = r.groupByKey()
grp_k.map(lambda x:(x[0], list(x[1]))).take(4)

[('a', [1, 1, 1, 1]), ('e', [1, 1, 1, 1, 1]), ('i', [1]), ('m', [1])]

In [205]:
reviews = sc.textFile('amazon_reviews.csv')
reviews.collect()

[u',id,name,username',
 u'0,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Cristina M',
 u'1,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Ricky',
 u'2,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Tedd Gardiner',
 u'3,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Dougal',
 u'4,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Miljan David Tanic',
 u'5,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Kelvin Law',
 u'6,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Ricky',
 u'7,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Bandler',
 u'8,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Cristina M',
 u'9,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Tedd Gardiner',
 u'10,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Miguel Martinez',
 u'11,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Magnus Brattemark',
 u'12,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Tedd Gardiner',
 u'13,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Janet Matthews Jan',
 u"14,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,John Kat's the brains",
 u'15,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,samira',
 u'16,AVpe7AsMilAPnD_xQ78G,Kindl

In [206]:
head = reviews.first()
header = sc.parallelize([head])
reviews = reviews.subtract(header)
reviews = reviews.map(lambda line: line.split(',')).map(lambda x: Row(blank = x[0], id = x[1],  product = x[2],  username = x[3]))

In [207]:
reviews.take(2)

[Row(blank=u'658', id=u'AVzRlpC2vKc47QAVd5FA', product=u'All-New Amazon Kid-Proof Case for Amazon Fire HD 8 Tablet (7th Generation', username=u'Betty Jaggers'),
 Row(blank=u'370', id=u'AVpfLiCSilAPnD_xWpk_', product=u'Amazon Fire TV', username=u'Gregory P. Baker')]

In [208]:
df = sql_c.createDataFrame(reviews)

In [209]:
df.show(2)

+-----+--------------------+--------------------+----------------+
|blank|                  id|             product|        username|
+-----+--------------------+--------------------+----------------+
|  658|AVzRlpC2vKc47QAVd5FA|All-New Amazon Ki...|   Betty Jaggers|
|  370|AVpfLiCSilAPnD_xWpk_|      Amazon Fire TV|Gregory P. Baker|
+-----+--------------------+--------------------+----------------+
only showing top 2 rows



In [321]:
df.select('product','username').groupby('product').count().sort(desc("count")).show()

+--------------------+-----+
|             product|count|
+--------------------+-----+
|Amazon Tap - Alex...|  542|
|      Amazon Fire TV|  166|
|Amazon Premium He...|  133|
|    Fire HD 6 Tablet|   87|
|"Kindle Fire HDX ...|   53|
|"Kindle Fire HDX ...|   43|
|"Kindle Fire HD 7"""|   41|
|   Kindle Paperwhite|   39|
|Certified Refurbi...|   38|
|     Kindle Keyboard|   32|
|All-New Amazon Fi...|   27|
|              Kindle|   20|
|Amazon 5W USB Off...|   19|
|All-New Amazon Fi...|   18|
|Replacement Remot...|   17|
|Echo Dot (2nd Gen...|   13|
|All-New Amazon Ki...|   13|
|Moshi Anti-Glare ...|   12|
|All-New Amazon Ki...|   12|
|Alexa Voice Remot...|   12|
+--------------------+-----+
only showing top 20 rows



In [319]:
df.select('product','username').groupby('username', 'product').count().sort(desc("count")).show()

+-------------------+--------------------+-----+
|           username|             product|count|
+-------------------+--------------------+-----+
|          A. Younan|Amazon Premium He...|   59|
|             Andrew|Amazon Premium He...|   43|
|          Victor L.|Amazon Premium He...|   30|
|     William Hardin|    Fire HD 6 Tablet|   30|
|            Mike W.|    Fire HD 6 Tablet|   29|
|      Earthling1984|    Fire HD 6 Tablet|   28|
|         B. Tarbuck|"Kindle Fire HDX ...|   16|
|     William Hardin|      Amazon Fire TV|   16|
|              Mandy|      Amazon Fire TV|   16|
|                 NF|"Kindle Fire HDX ...|   15|
|                 NF|"Kindle Fire HD 7"""|   14|
|    Amazon Reviewer|"Kindle Fire HDX ...|   14|
|    Amazon Reviewer|"Kindle Fire HDX ...|   13|
|     William Hardin|Certified Refurbi...|   12|
|  Michael Gallagher|"Kindle Fire HDX ...|   12|
|             Dallas|      Amazon Fire TV|   12|
|   Gregory P. Baker|      Amazon Fire TV|   11|
|              MarkM

In [210]:
df2 = df['product', 'username']
rdd = df2.rdd.map(tuple)
rdd = rdd.map(lambda x: (x[0].replace('"',''), 1))
rdd.collect()
# user_to_prod = rdd.groupByKey()
# user_to_prod.collect()

[(u'All-New Amazon Kid-Proof Case for Amazon Fire HD 8 Tablet (7th Generation',
  1),
 (u'Amazon Fire TV', 1),
 (u'Amazon Premium Headphones', 1),
 (u'Fire Kids Edition Tablet', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'Alexa Voice Remote for Amazon Echo and Echo Dot', 1),
 (u'Amazon Fire TV', 1),
 (u'Kindle Fire HDX 7', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (7th Generation',
  1),
 (u'Amazon Premium Headphones', 1),
 (u'Amazon Premium Headphones', 1),
 (u'Amazon Fire TV', 1),
 (u'Fire HD 8 Tablet', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'Amazon Fire TV', 1),
 (u'Kindle Fire HDX 7', 1),
 (u'Amazon Fire TV', 1),
 (u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 1),
 (u'Certified Refurbished Amazon Fire 

In [340]:
# user_prod_list = user_to_prod.map(lambda x: (x[0].replace('"',''), len(list(x[1]))))
# user_prod_list.sortBy(lambda x: x[1], ascending=False).take(10)
rdd.reduceByKey(lambda v1,v2: v1 + v2).sortBy(lambda x: x[1], ascending=False).take(5)

[(u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 542),
 (u'Amazon Fire TV', 166),
 (u'Amazon Premium Headphones', 133),
 (u'Fire HD 6 Tablet', 87),
 (u'Kindle Fire HDX 7', 53)]

In [297]:
df3 = df['username', 'product']
df3 = df3[df3['username'] != 'Amazon Customer'] #remove generic username
user_prod = df3.rdd.map(tuple)
user_prod = user_prod.groupByKey()
user_prod.collect()

[(u'Pete', <pyspark.resultiterable.ResultIterable at 0x10b8f6110>),
 (u'Blue', <pyspark.resultiterable.ResultIterable at 0x10b907d10>),
 (u'In4dfun2', <pyspark.resultiterable.ResultIterable at 0x10b907810>),
 (u'Harper', <pyspark.resultiterable.ResultIterable at 0x10b907b50>),
 (u'Hunter', <pyspark.resultiterable.ResultIterable at 0x10b907bd0>),
 (u'Tinkerer', <pyspark.resultiterable.ResultIterable at 0x10b907850>),
 (u'agzela', <pyspark.resultiterable.ResultIterable at 0x10b907c10>),
 (u'Czer', <pyspark.resultiterable.ResultIterable at 0x10b907a50>),
 (u'SBgamer', <pyspark.resultiterable.ResultIterable at 0x10b907ad0>),
 (u'Tracy', <pyspark.resultiterable.ResultIterable at 0x10b907a90>),
 (u'BigToe', <pyspark.resultiterable.ResultIterable at 0x10b907990>),
 (u'Pita', <pyspark.resultiterable.ResultIterable at 0x10b9078d0>),
 (u'AAAC', <pyspark.resultiterable.ResultIterable at 0x10b907a10>),
 (u'Missy', <pyspark.resultiterable.ResultIterable at 0x10b907910>),
 (u'Phobia3000', <pyspark.r

In [298]:
# user_prod_list = user_prod.map(lambda x: (x[0], list(x[1]))).sortBy(lambda x: len(x[1]), ascending=False).collect()
user_prod_list = user_prod.map(lambda x: (x[0], [p.replace('"','') for p in list(x[1])]))
user_prod_list.sortBy(lambda x: len(x[1]), ascending=False).take(10)

[(u'A. Younan',
  [u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headphones',
   u'Amazon Premium Headpho

In [337]:
user_prod_list_count = user_prod_list.map(lambda x: (x[0],(dict(Counter(x[1])), len(x[1]))))
user_prod_list_count.take(10)

[(u'Pete', ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Blue', ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'In4dfun2',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Harper',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Hunter',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Tinkerer',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'agzela',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Czer', ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'SBgamer',
  ({u'Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker': 1}, 1)),
 (u'Tracy', ({u'Kindle': 1}, 1))]

In [339]:
user_prod_list_count.sortBy(lambda x: x[1][1], ascending=False).take(3)

[(u'A. Younan', ({u'Amazon Premium Headphones': 59}, 59)),
 (u'William Hardin',
  ({u'Amazon Fire TV': 16,
    u'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)': 12,
    u'Fire HD 6 Tablet': 30},
   58)),
 (u'Andrew', ({u'Amazon Premium Headphones': 43}, 43))]